<a href="https://colab.research.google.com/github/pavansai26/TPU-TRAINING/blob/main/TPU_TRAINING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

Before you run this Colab notebook, make sure that your hardware accelerator is a TPU by checking your notebook settings: Runtime > Change runtime type > Hardware accelerator > TPU.

Import some necessary libraries

In [1]:
import tensorflow as tf

import os
import tensorflow_datasets as tfds

TPU initialization

TPUs are typically Cloud TPU workers, which are different from the local process running the user's Python program.

Thus, you need to do some initialization work to connect to the remote cluster and initialize the TPUs. Note that the tpu argument to tf.distribute.cluster_resolver.TPUClusterResolver is a special address just for Colab.

Note: The TPU initialization code has to be at the beginning of your program.


In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


device placement

After the TPU is initialized, you can use manual device placement to place the computation on a single TPU device

In [3]:
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

with tf.device('/TPU:0'):
  c = tf.matmul(a, b)

print("c device: ", c.device)
print(c)

c device:  /job:worker/replica:0/task:0/device:TPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


Distribution strategies

Usually, you run your model on multiple TPUs in a data-parallel way. To distribute your model on multiple TPUs (as well as multiple GPUs or multiple machines), TensorFlow offers the tf.distribute.Strategy API.

Using the tf.distribute.TPUStrategy option implements synchronous distributed training.

In [5]:
strategy = tf.distribute.TPUStrategy(resolver)
strategy


Classification on TPUs

This section demonstrates how to use the distribution strategy—tf.distribute.TPUStrategy—to train a Keras model on a Cloud TPU.

In [6]:
def get_dataset(batch_size, is_training=True):
  split = 'train' if is_training else 'test'
  dataset, info = tfds.load(name='mnist', split=split, with_info=True,
                            as_supervised=True, try_gcs=True)

  # Normalize the input data.
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label

  dataset = dataset.map(scale)

  # Only shuffle and repeat the dataset in training. The advantage of having an
  # infinite dataset for training is to avoid the potential last partial batch
  # in each epoch, so that you don't need to think about scaling the gradients
  # based on the actual batch size.
  if is_training:
    dataset = dataset.shuffle(10000)
    dataset = dataset.repeat()

  dataset = dataset.batch(batch_size)

  return dataset

Start with a definition of a Sequential Keras model for image classification on the MNIST dataset. It's no different than what you would use if you were training on CPUs

Note that Keras model creation needs to be inside the Strategy.scope, so the variables can be created on each TPU device. Other parts of the code are not necessary to be inside the Strategy scope.

In [8]:
def create_model():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
       tf.keras.layers.Conv2D(256, 3, activation='relu'),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(256, activation='relu'),
       tf.keras.layers.Dense(128, activation='relu'),
       tf.keras.layers.Dense(10)])

Train the model using Keras high-level APIs

You can train your model with Keras Model.fit and Model.compile APIs. There is nothing TPU-specific

In [9]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

batch_size = 200
steps_per_epoch = 60000 // batch_size
validation_steps = 10000 // batch_size

train_dataset = get_dataset(batch_size, is_training=True)
test_dataset = get_dataset(batch_size, is_training=False)

model.fit(train_dataset,
          epochs=5,
          steps_per_epoch=steps_per_epoch,
          validation_data=test_dataset,
          validation_steps=validation_steps)

Epoch 1/5
300/300 [==============================] - 21s 43ms/step - loss: 0.1390 - sparse_categorical_accuracy: 0.9565 - val_loss: 0.0547 - val_sparse_categorical_accuracy: 0.9835
Epoch 2/5
300/300 [==============================] - 10s 34ms/step - loss: 0.0335 - sparse_categorical_accuracy: 0.9901 - val_loss: 0.0439 - val_sparse_categorical_accuracy: 0.9865
Epoch 3/5
300/300 [==============================] - 10s 35ms/step - loss: 0.0194 - sparse_categorical_accuracy: 0.9938 - val_loss: 0.0401 - val_sparse_categorical_accuracy: 0.9881
Epoch 4/5
300/300 [==============================] - 11s 37ms/step - loss: 0.0117 - sparse_categorical_accuracy: 0.9962 - val_loss: 0.0511 - val_sparse_categorical_accuracy: 0.9855
Epoch 5/5
300/300 [==============================] - 11s 35ms/step - loss: 0.0081 - sparse_categorical_accuracy: 0.9973 - val_loss: 0.0644 - val_sparse_categorical_accuracy: 0.9843


To reduce Python overhead and maximize the performance of your TPU, pass in the steps_per_execution argument to Keras Model.compile. In this example, it increases throughput by about 50%:

In [10]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                # Anything between 2 and `steps_per_epoch` could help here.
                steps_per_execution = 50,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

model.fit(train_dataset,
          epochs=5,
          steps_per_epoch=steps_per_epoch,
          validation_data=test_dataset,
          validation_steps=validation_steps)

Epoch 1/5
300/300 [==============================] - 17s 56ms/step - loss: 0.1411 - sparse_categorical_accuracy: 0.9566 - val_loss: 0.0453 - val_sparse_categorical_accuracy: 0.9856
Epoch 2/5
300/300 [==============================] - 5s 16ms/step - loss: 0.0342 - sparse_categorical_accuracy: 0.9896 - val_loss: 0.0343 - val_sparse_categorical_accuracy: 0.9888
Epoch 3/5
300/300 [==============================] - 4s 14ms/step - loss: 0.0192 - sparse_categorical_accuracy: 0.9935 - val_loss: 0.0460 - val_sparse_categorical_accuracy: 0.9860
Epoch 4/5
300/300 [==============================] - 4s 14ms/step - loss: 0.0124 - sparse_categorical_accuracy: 0.9960 - val_loss: 0.0379 - val_sparse_categorical_accuracy: 0.9884
Epoch 5/5
300/300 [==============================] - 4s 15ms/step - loss: 0.0098 - sparse_categorical_accuracy: 0.9970 - val_loss: 0.0456 - val_sparse_categorical_accuracy: 0.9852
